In [2]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import TypedDict, Literal
from pydantic import BaseModel, Field

In [ ]:
load_dotenv()

In [ ]:
model = ChatOpenAI(model = 'gpt-4o-mini')

In [ ]:
class SentimentSchema(BaseModel):

    sentiment : Literal['positive', 'negative'] = Field(description='Sentiment of the review')

In [ ]:
structured_model = model.with_structured_output(SentimentSchema)

prompt = 'what is the sentiment of the following review - The software is too bad'

structured_model.invoke(prompt).sentiment

In [3]:
class ReviewState(TypedDict):

    review : str
    sentiment : str = Literal['positive', 'negative']
    diagnosis : dict 
    response : str

In [ ]:
def find_sentiment(state: ReviewState):

    prompt = f'for the following review find out the sentiment \n {state['review']}'

    sentiment = structured_model.invoke(prompt).sentiment

    return {'sentiment': sentiment}



In [ ]:
graph = StateGraph(ReviewState)

graph.add_node('find_sentiment', find_sentiment)

graph.add_edge(START, 'find_sentiment')
graph.add_edge('find_sentiment', END)

workflow = graph.compile()

In [ ]:
initial_state = {
    'review' : 'The product was really good'
}

workflow.invoke(initial_state)